##### demo1 像素值替换重新上色

In [4]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
Image.MAX_IMAGE_PIXELS = None

def pixel_grayscale(image_file,save_file1,save_file2):
    if not os.path.exists(save_file1):
        os.makedirs(save_file1)
    if not os.path.exists(save_file2):
        os.makedirs(save_file2)
    for name in tqdm.tqdm(os.listdir(image_file)):
        save_voc = os.path.join(save_file1,name)
        save_path = os.path.join(save_file2,name.split('.')[0]+'.jpg')
        img = cv2.imread(os.path.join(image_file,name))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        h,w, = img.shape
        b = np.empty([h,w])
        b[np.where(img>249)]=1
        b[np.where((90>img)&(img>70))] =2
        b[img==150] =3
        cv2.imwrite(save_voc,b)
        b = Image.fromarray(b)
        b = b.convert('L')
        #调色板
        palette = [0, 0, 0,0, 255, 0, 255, 0, 0,255,255,255]
        #着色
        b.putpalette(palette)
        b.save(save_path)
image_file  = r"D:\31890\Desktop\biaozhuchuli"
save_file1  = r"D:\31890\Desktop\2"
save_file2  = r"D:\31890\Desktop\3"
pixel_grayscale(image_file,save_file1,save_file2)

100%|██████████| 12/12 [00:48<00:00,  4.00s/it]


##### demo2 聚类 

In [ ]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
Image.MAX_IMAGE_PIXELS = None
img = cv2.imread(r"D:\software\Code\codefile\m_CV2\image\102_cmask_sen.jpg")
#展平
image_flat = img.reshape(-1,3)
image_flat = np.float32(image_flat)
    #终止条件
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_MAX_ITER, 20, 0.5)
    #flags
flags = cv2.KMEANS_RANDOM_CENTERS
    #flags = cv2.KMEANS_PP_CENTERS
    #K-Means聚类 聚集成4类
compactness, labels, centers = cv2.kmeans(image_flat, 5, None, criteria, 10, flags)
    #labels.shape=((3408500, 1))      centers.shape = ((4, 3))
centers = np.uint8(centers)
res = centers[labels.flatten()]
save = res.reshape((img.shape))
cv2.imwrite(r'D:\software\Code\codefile\m_CV2\image\102_cmask_sen_k.png', save)

##### demo3 超像素

In [ ]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
Image.MAX_IMAGE_PIXELS = None
image_array = cv2.imread(r'D:\software\Code\codefile\m_CV2\image\102_cmask_sen.jpg')
slic = cv2.ximgproc.createSuperpixelSLIC(image_array,region_size=18,ruler = 40.0)
slic.iterate(10)                                    #迭代次数，越大效果越好
label_slic = slic.getLabels()                       #获取超像素标签
number_slic = slic.getNumberOfSuperpixels()         #获取超像素数目
color_array = np.zeros((number_slic,3))
for i in range(number_slic):
    temp_array = image_array[label_slic==(i)]
    temp_array_mean = np.mean(temp_array,axis=0)
    color_array[i]=temp_array_mean
color_array = color_array.astype(np.int32)
result = color_array[(label_slic).flatten()]
result = result.reshape(image_array.shape)
cv2.imwrite(r'D:\software\Code\codefile\m_CV2\image\102_cmask_sen_SLIC.png')

##### demo4 分割结果间对比，类qs

In [ ]:
import cv2,glob,os,tqdm
import numpy as np
from PIL import Image

def fun(ground_true,file_list,o):
    if not os.path.exists(o):
        os.makedirs(o)
    for p in tqdm.tqdm(file_list):
        for name in tqdm.tqdm(os.listdir(ground_true)):
            img1_p = os.path.join(ground_true,name)
            img2_P = os.path.join(p,name)
            img1 = cv2.imread(img1_p,cv2.IMREAD_GRAYSCALE)
            img2 = cv2.imread(img2_P,cv2.IMREAD_GRAYSCALE)

            # cv2.imwrite(os.path.join(o,name.split('.')[0]+p.split('\\')[-1]+'.png'),img2)

            img1 = np.where(img1!=0,1,0)
            img2 = np.where(img2!=0,2,0)

            imgs_union = img1+img2

            imgs_union = Image.fromarray(imgs_union)

            imgs_union = imgs_union.convert('L')
                    #调色板
            palette = [0, 0, 0,50, 205, 50, 175, 34, 34,255,255,255]  #
                    # [0, 0, 0, 0, 0, 128, 255, 165, 0,255,255,255] 
                    #0 背景 1 标注有，预测没有 2标注没有，预测有 3 两者交集
                    #着色
            imgs_union.putpalette(palette)
            save_path = os.path.join(o,name.split('.')[0]+p.split('\\')[-1]+'_duibi'+'.png')
            imgs_union.save(save_path)

file_list  =glob.glob(r"D:\31890\Desktop\tranformer\model_compare\pred\*")
ground_true = r"D:\31890\Desktop\tranformer\model_compare\segformb5"
o = r"D:\31890\Desktop\tranformer\model_compare\segduibi"
fun(ground_true,file_list,o)

##### demo5 分割可视化

In [ ]:
# 
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
def fuse(img_p1,img_p2,save_p):
    image1 = Image.open(img_p1)#image1 原图 
    image2 = Image.open(img_p2)#image2 分割图

    image2.convert('L')
    palette = [0, 0, 0,50, 205, 50, 175, 34, 34,255,255,255]
    # [0, 0, 0,50, 205, 50, 175, 34, 34,255,255,255]衰老标注
    #
    image2.putpalette(palette)
    
    image1 = image1.convert('RGBA')
    image2 = image2.convert('RGBA')
    
    #两幅图像进行合并时，按公式：blended_img = img1 * (1 – alpha) + img2* alpha 进行
    image = Image.blend(image1,image2,0.3)

    image.save(save_p)
    # image.show()
img_p1 = r'D:\31890\Desktop\codefile\Utils\Image\yuantu.jpg'
img_p2 = r'D:\31890\Desktop\codefile\Utils\Image\101.png'
save_p = r'D:\31890\Desktop\codefile\Utils\Image\shuailao.png'
fuse(img_p1,img_p2,save_p)

In [ ]:
import os,cv2,tqdm
import numpy as np
def fun(i,o):
    if not os.path.exists(o):
        os.makedirs(o)
    for name in tqdm.tqdm(os.listdir(i)):
        img_ph = os.path.join(i,name)
        img = cv2.imread(img_ph,cv2.IMREAD_GRAYSCALE)
        img = np.where(img!=0,255,0)
        cv2.imwrite(os.path.join(o,name),img)

i = r'D:\31890\Desktop\codefile\result\Train_data\sen_data\anno\train'
o = r'D:\31890\Desktop\codefile\result\Train_data\sen_data\anno\train2'
fun(i,o)

In [ ]:
import os,tqdm,cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.segmentation import find_boundaries


def Superpixel_segmentation(image_array,mask_array):
    #初始化slic项，超像素平均尺寸20（默认为10），平滑因子20
    slic = cv2.ximgproc.createSuperpixelSLIC(image_array,region_size=120,ruler = 20)
    slic.iterate(50)                                    #迭代次数，越大效果越好
    mask_slic = slic.getLabelContourMask()              #获取Mask，超像素边缘Mask==1
    label_slic = slic.getLabels()                       #获取超像素标签矩阵
    label_slic = cv2.bitwise_and(label_slic, label_slic, mask=mask_array).astype(np.uint8)

    class_num = np.unique(label_slic,return_index=False,return_inverse=False)
    for i in range(len(class_num)):
        label_slic[np.where(label_slic==class_num[i])] = i

    mask_slic = find_boundaries(label_slic,mode='inner').astype(np.uint8)
    mask_inv_slic = cv2.bitwise_not((mask_slic*255).astype(np.uint8))          #获取分类边界

    img_slic = cv2.bitwise_and(image_array,image_array,mask =  mask_inv_slic)  #在原图上绘制超像素边界
    # cv2.imshow("img_slic",img_slic)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img_slic

def Superpixel_averaging(image_array):
    #初始化slic项，超像素平均尺寸20（默认为10），平滑因子20
    slic = cv2.ximgproc.createSuperpixelSLIC(image_array,region_size=120,ruler = 40.0)
    slic.iterate(10)                                    #迭代次数，越大效果越好
    label_slic = slic.getLabels()                       #获取超像素标签
    number_slic = slic.getNumberOfSuperpixels()         #获取超像素数目
    color_array = np.zeros((number_slic,3))
    for i in range(number_slic):
        temp_array = image_array[label_slic==(i)]
        temp_array_mean = np.mean(temp_array,axis=0)
        color_array[i]=temp_array_mean
    color_array = color_array.astype(np.int32)
    result = color_array[(label_slic).flatten()]
    result = result.reshape(image_array.shape)
    return result

image_array = cv2.imread(r"D:\31890\Desktop\codefile\Utils\Image\2-2019-7-07-1200_8_13_.png")
mask_array = cv2.imread(r'D:\31890\Desktop\codefile\Utils\Image\2-2019-7-07-1200_8_13.png',cv2.IMREAD_GRAYSCALE)
result = Superpixel_segmentation(image_array,mask_array)
cv2.imwrite(r"D:\31890\Desktop\codefile\Utils\Image\2.png",result)

##### demo6 根系的主成分分析加聚类结果

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.cluster 
import pandas as pd
from time import time # 用于计算运行时间
import numpy as np
from matplotlib.pyplot import MultipleLocator
from sklearn import decomposition

def plot_embedding(X, img_class,title=None):              ####—————— 首先定义函数画出二维空间中的样本点，输入参数：1.降维后的数据；2.图片标题
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)           ####—————— 对每一个维度进行0-1归一化，注意此时X只有两个维度
    
    plt.figure(figsize= (18,18))                ####—————— 设置整个图形大小
    ax = plt.subplot(111)
    
    img_class_lits=[]
    for i in img_class:
        if i not in img_class_lits:
            img_class_lits.append(i)
    for i,j in enumerate(img_class_lits):
        img_class[img_class==j] =i+10
    img_class = img_class-10
    colors = [  '#FF0000','#FF8C00','#FFE4B5','#9ACD32','#00FF00',      ####—————— 设置颜色矩阵
                '#00FFFF','#4169E1','#9400D3','#FF00FF','#708090']
    legend_list=[]
    # a,b, c= np.unique(img_class,return_index=True,return_counts=True)
    # for i in range(len(a)):
        
    for i in range(10):
        postion = np.where(img_class == i)[0]
        plt.scatter(X[postion[0]:postion[-1]+1, 0], X[postion[0]:postion[-1]+1, 1],c=colors[i])
        legend_list.append('day{}--day{}'.format(postion[0]+1,postion[-1]+1))

    for i in range(X.shape[0]):                                         ####—————— 每一行代表一个样本
        plt.text(X[i, 0]+0.01, X[i, 1], str(i+1),
                 color='#A68064',
                 fontdict={'weight': 'bold', 'size': 12})               ####—————— 在样本点所在位置画出样本点的数字标签
    
    plt.xlim(-0.1,1.1)                              ####—————— 设置x轴的刻度限制
    plt.ylim(-0.1,1.1)                              ####—————— 设置y轴的刻度限制

    y_major_locator=MultipleLocator(0.1)            ####—————— 把y轴的刻度间隔设置为10，并存在变量里

    ax=plt.gca()                                    ####——————  ax为两条坐标轴的实例
    ax.yaxis.set_major_locator(y_major_locator)

    plt.legend(legend_list, loc='upper right',prop = {'size':16
    })
    
    if title is not None: 
        plt.title(title)                            ####—————— 设置图标的标题
    plt.savefig(r"D:\31890\Desktop\codefile\Utils\result\shuailao_cluster.svg",dpi=200, bbox_inches='tight')
    plt.show()

path = r'D:\31890\Desktop\codefile\Utils\result\shuailao pixel statistics.xlsx'
data_X = pd.read_excel(path)                ####—————— 读取excel
data_X_value = data_X.values                ####—————— 转为矩阵
img_flatten_group = data_X_value[:,2:]      
img_name_list = data_X_value[:,0].flatten().tolist()

t0 = time()                                 ####—————— 计时
pca = decomposition.PCA(n_components=2)     ####—————— 降维分析(PCA,NMF,ICA)
X_pca = pca.fit_transform(img_flatten_group.astype(np.float32))
cluster_result = sklearn.cluster.AgglomerativeClustering(n_clusters=10).fit(X_pca)      ####—————— 聚类 (PCA,NMF,ICA)
img_class = cluster_result.labels_          ####—————— 获取聚类类别

df2 = pd.DataFrame(X_pca,columns=['x','y'])
df2['class']=img_class
df2.to_csv(r'D:\31890\Desktop\codefile\Utils\result\pca_r.csv')       ####—————— 保存类别

plot_embedding(X_pca, img_class,title=None)


##### demo7 黑白反转

In [6]:
import cv2,os,tqdm
import numpy as np

def Black_white_conversion(array):      ##   黑白转换
    array = np.where(array==0,256,1)
    return array-1

image_file = r'D:\31890\Desktop\codefile\Utils\time_seris\mask_heidi'
save_file =  r'D:\31890\Desktop\codefile\Utils\time_seris\mask_heidi_conversion'

if not os.path.exists(save_file):
    os.makedirs(save_file)

for name in tqdm.tqdm(os.listdir(image_file)):
    array = cv2.imread(os.path.join(image_file,name),cv2.IMREAD_GRAYSCALE) 
    array = Black_white_conversion(array)
    cv2.imwrite(os.path.join(save_file,name),array)

100%|██████████| 102/102 [03:14<00:00,  1.91s/it]


##### demo8 衰老和正常根系提取，变换为表型检测格式

In [9]:
import cv2,os,tqdm
import numpy as np
image_file = r'D:\31890\Desktop\codefile\Utils\time_seris\shuailao\2-2019-8-06-1200.png'

array = cv2.imread(image_file,cv2.IMREAD_GRAYSCALE)
h,w = array.shape
normal_array = np.ones((h,w))*255
senescence_array = np.ones((h,w))*255
taproot_array = np.ones((h,w))*255

normal_array[array==140]=0
senescence_array[array==76]=0
taproot_array[array==255]=0

cv2.imwrite(r'D:\31890\Desktop\codefile\Utils\normal_array.png',normal_array)
cv2.imwrite(r'D:\31890\Desktop\codefile\Utils\senescence_array.png',senescence_array)
cv2.imwrite(r'D:\31890\Desktop\codefile\Utils\taproot_array.png',taproot_array)

True

##### demo9 resize原图 加 生成黑白

In [2]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
Image.MAX_IMAGE_PIXELS = None

def pixel_grayscale(image_file,o,o_r,h_f):
    if not os.path.exists(o_r):
        os.makedirs(o_r)
    if not os.path.exists(h_f):
        os.makedirs(h_f)
    for name in tqdm.tqdm(os.listdir(image_file)):
        o_rp = os.path.join(o_r,name.split('.')[0]+'.jpg')
        hp = os.path.join(h_f,name)
        img1 = cv2.imread(os.path.join(image_file,name),cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(os.path.join(o,name.split('.')[0]+'.jpg'))

        h,w, = img1.shape
        img2 = cv2.resize(img2,(w,h))
        cv2.imwrite(o_rp,img2)

        img1[img1!=0]=255
        cv2.imwrite(hp,img1)



image_file  = r"D:\31890\Desktop\1"
o  = r"D:\31890\Desktop\o"
o_r  = r"D:\31890\Desktop\o2r"
h  = r"D:\31890\Desktop\heibai"
pixel_grayscale(image_file,o,o_r,h)

100%|██████████| 12/12 [00:57<00:00,  4.79s/it]


##### demo10 无监督分割鸡蛋

In [ ]:
import cv2,os,sklearn,tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans,AgglomerativeClustering,MeanShift,SpectralClustering
from utils import color
from PIL import Image


def Clustering(img_array,method="kmeans"):
    h,w,c = img_array.shape

    img_array =  img_array.reshape(-1,3).astype(np.uint8)
    if method == "kmeans":
        classifier = KMeans(n_clusters=2,n_init="auto",random_state=0,init='k-means++')
    elif method == "AgglomerativeClustering":               # 不适用,计算量太大
        classifier = AgglomerativeClustering(n_clusters=2)
    elif method == "MeanShift":                             # 耗时过长
        classifier = MeanShift(bandwidth=2)
    elif method == "SpectralClustering":                    # 不适用,计算量太大
        classifier = SpectralClustering(n_clusters=2)
        
    kmeans = classifier.fit(img_array)
    img_label = kmeans.labels_
    img_label = img_label.reshape(h,w)

    return img_label


img_file = r"D:\31890\Desktop\codefile\Utils\data"
save_file = r'D:\31890\Desktop\codefile\Utils\result'

if not os.path.exists(save_file):
    os.makedirs(save_file)

for name in tqdm.tqdm(os.listdir(img_file)):
    image_path = os.path.join(img_file,name)
    save_path = os.path.join(save_file,name)

    image_array = cv2.imread(image_path)
    img_label = Clustering(image_array,method="kmeans")
    img_label = color(img_label,colormap=[0, 0, 0,255,255,255])
    img_label.save(save_path)

##### demo11 画椭圆

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
from numpy.random import randint as randint
 
def ellipse(img):
    h,w,c = img.shape

    center_x = randint(w//2-10, high=w//2+10)  #随机绘制圆心
    center_y = randint(h//2-10, high=h//2+10)
    X = randint(360, 380)//2
    Y = randint(460, 480)//2
    # 参数 1.目标图片  2.椭圆圆心  3.长短轴长度  4.偏转角度  5.圆弧起始角度  6.终止角度  7.颜色  8.是否填充
    cv2.ellipse(img, (center_x,center_y), (X,Y), randint(0,120), 0, 360, (255, 255, 255),-1)

def main():
    # 3.显示结果
    number =15
    max_shape=900
    min_shape=670
    for i in range(number):
        img = np.ones((randint(min_shape, max_shape),randint(min_shape, max_shape) , 3), np.uint8)*0
        ellipse(img)
 
        imgpath = './fig/' + str(i) + '.jpg'
        cv2.imwrite(imgpath, img)

##### demo12 黑白mask的24位图转8位图


In [1]:
import cv2,os,tqdm
import numpy as np

def pixel_grayscale(image_file,save_file):
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    for name in tqdm.tqdm(os.listdir(image_file)):
        save_path = os.path.join(save_file,name)
        img = cv2.imread(os.path.join(image_file,name),cv2.IMREAD_GRAYSCALE)
        img = np.where(img>10,255,0)
        
        cv2.imwrite(save_path,img)
image_file = r'D:\31890\Desktop\data\anno\train'
save_file  = r'D:\31890\Desktop\data\anno\train_8'
pixel_grayscale(image_file,save_file)

100%|██████████| 18166/18166 [02:17<00:00, 132.03it/s]


##### demo13  8位黑白图转为voc格式

In [9]:
def pixel_replacement(image_file,save_file):
    for name in tqdm.tqdm(os.listdir(image_file)):
        save_path = os.path.join(save_file,name)
        image_path = os.path.join(image_file,name)
        img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        img2 = np.where(img>0,1,0)
        cv2.imwrite(save_path,img2)

image_file = r'D:\31890\Desktop\data\anno\test'
save_file  = r'D:\31890\Desktop\data\anno\test_voc'
pixel_replacement(image_file,save_file)

  0%|          | 0/14100 [00:00<?, ?it/s]

100%|██████████| 14100/14100 [01:47<00:00, 131.70it/s]


##### 图像添加黑边方式

In [ ]:
# 利用矩阵
# 新的高和宽
new_h = int((origin_h//pixel_shape+1)*pixel_shape)
new_w = int((origin_w//pixel_shape+1)*pixel_shape)
# 多余的尺寸
h_padding = new_h-origin_h
w_padding = new_w-origin_w
# 创建新的矩阵
img_b = np.zeros((new_h, new_w, c))
mask = np.zeros((c,new_h, new_w))
img_b[:origin_h, :origin_w, :] = img_array


# 利用pillow
img_new = Image.new('RGB', (w, h), color)
img_new.paste(img_ori, (0, width, w, h))